### Sort Snippets by topic_id

Core Idea:
1. Sort snippets by topic, then remove redundant snippets
2. Keep 36 irrelevant snippets / 4 relevant snippets per topic_id

In [7]:
from loguru import logger

import pandas as pd

from slt_positional_bias.dataset import generate_merged_data_frame, sort_data_frame

df = generate_merged_data_frame()
df

2025-07-22 23:27:23.410 | INFO     | slt_positional_bias.dataset:generate_data_frame:83 - file path exists
2025-07-22 23:27:28.039 | INFO     | slt_positional_bias.dataset:generate_data_frame:83 - file path exists
2025-07-22 23:27:28.044 | INFO     | slt_positional_bias.dataset:generate_data_frame:83 - file path exists
2025-07-22 23:27:28.246 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:70 - df column 'doc_id' is unique: True
2025-07-22 23:27:28.252 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:71 - df contains NaN: False


,topic_id,doc_id,rel_scoring,topic,doc
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
29158,2024-127266,msmarco_v2.1_doc_58_1230815815#5_1795053441,2,what are some key challenges related to the re...,Highest E-Waste Generating Nations In The Worl...
29159,2024-127266,msmarco_v2.1_doc_34_470264225#3_1018882699,0,what are some key challenges related to the re...,The Pros & Cons Of Landfills (Benefits & Disad...
29160,2024-127266,msmarco_v2.1_doc_57_781229369#6_1221950031,1,what are some key challenges related to the re...,The Patron Saint of Electronics? Do you Know t...
29161,2024-127266,msmarco_v2.1_doc_57_781235684#6_1221962946,1,what are some key challenges related to the re...,Destroy VHS Tapes \nDestroy VHS Tapes\nDaliah ...


In [8]:
counts = df["topic_id"].value_counts()
counts

topic_id
2024-133810    622
2024-36302     551
2024-141577    514
2024-224279    510
2024-44059     506
              ... 
2024-129925    173
2024-224701    173
2024-149459    170
2024-42497     159
2024-145979    153
Name: count, Length: 89, dtype: int64

In [9]:
# df_rel_0 = (
#     df[df["rel_scoring"] == 0]
#     .groupby("topic_id", group_keys=False)
#     .apply(lambda g: g.head(36))
# )
# 
# df_rel_0

In [10]:
# Group by topic_id
groups = df.groupby("topic_id", group_keys=False)

# Filter out all topic_ids with count(rel_scoring==0)<36 && count(rel_scoring==3)<4
valid_topic_ids = [
    topic for topic, group in groups
    if (group["rel_scoring"] == 0).sum() >= 36 and (group["rel_scoring"] == 3).sum() >= 4
]

df_filtered = df[df["topic_id"].isin(valid_topic_ids)]

grouped = df_filtered.groupby("topic_id", group_keys=False)

df_filtered_rel = grouped.apply(
    lambda g: pd.concat([
        g[g["rel_scoring"] == 0].head(36),
        g[g["rel_scoring"] == 3].head(4)
    ])
)

df_filtered_rel = df_filtered_rel.reset_index(drop=True)
df_filtered_rel

C:\Users\Albert\AppData\Local\Temp\ipykernel_7980\1943989705.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered_rel = grouped.apply(


,topic_id,doc_id,rel_scoring,topic,doc
0,2024-127266,msmarco_v2.1_doc_51_1637162254#13_3136723187,0,what are some key challenges related to the re...,"Solid waste issue: Sources, composition, dispo..."
1,2024-127266,msmarco_v2.1_doc_08_564630335#9_1002896073,0,what are some key challenges related to the re...,\nImportance of Recycling Plastic\nTypes of P...
2,2024-127266,msmarco_v2.1_doc_19_1124745018#2_1589311387,0,what are some key challenges related to the re...,Trending Recycling Essay Topics for You to Not...
3,2024-127266,msmarco_v2.1_doc_54_361376333#16_839575849,0,what are some key challenges related to the re...,E-Waste Recycling Facts and Figures E-Waste Re...
4,2024-127266,msmarco_v2.1_doc_51_877973062#2_1847163672,0,what are some key challenges related to the re...,How to recycle electronics in Chicago: A guide...
...,...,...,...,...,...
1595,2024-96063,msmarco_v2.1_doc_25_947435380#15_1757291433,0,how using maps can impact your pedagogy,Intersectional Pedagogy | Denver Intersectiona...
1596,2024-96063,msmarco_v2.1_doc_52_1260126505#8_2526623662,3,how using maps can impact your pedagogy,Maps and Map Learning in Social Studies\nRes...
1597,2024-96063,msmarco_v2.1_doc_33_1002533346#9_2137815825,3,how using maps can impact your pedagogy,Why mind mapping works: the benefits of mind m...
1598,2024-96063,msmarco_v2.1_doc_14_1335772831#2_2728776902,3,how using maps can impact your pedagogy,8 Visual Teaching Strategies to Win Over Your ...


In [11]:
rel_3 = df_filtered_rel[df_filtered_rel["rel_scoring"] == 3]
counts = rel_3["topic_id"].value_counts()
least_topic_id = counts.idxmin()
least_count = counts.min()

logger.info(f"topic {least_topic_id} has {least_count} docs rated rel 3")



2025-07-22 23:27:28.572 | INFO     | __main__:<module>:6 - topic 2024-127266 has 4 docs rated rel 3
